# Scenario Studio – Fuel Bus Demo
Run PF with explicit fuel buses. Adjust the YAMLs to change topology/params.


In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🔥 EnerGIS Scenario Studio\n",
    "Interaktive Optimierung von industriellen Energiesystemen\n",
    "\n",
    "## 🎯 Quick Start\n",
    "1. Kernel auswählen (oben rechts): Python 3.11+ empfohlen\n",
    "2. Alle Zellen nacheinander mit **Shift+Enter** ausführen\n",
    "3. Bei Fehlern: Traceback lesen und Config/Daten prüfen\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📦 Setup & Imports"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# System-Path Setup (damit energis gefunden wird)\n",
    "import sys\n",
    "from pathlib import Path\n",
    "\n",
    "# Automatisch zum Projekt-Root navigieren\n",
    "notebook_path = Path().resolve()\n",
    "project_root = notebook_path.parent if notebook_path.name == 'notebooks' else notebook_path\n",
    "\n",
    "if str(project_root) not in sys.path:\n",
    "    sys.path.insert(0, str(project_root))\n",
    "\n",
    "print(f\"📁 Projekt-Root: {project_root}\")\n",
    "print(f\"✅ Python-Path erweitert\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Imports\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import warnings\n",
    "from datetime import datetime\n",
    "\n",
    "# EnerGIS-Module\n",
    "try:\n",
    "    from energis.run.orchestrator import run_all\n",
    "    from energis.config.merge import load_and_merge\n",
    "    print(\"✅ EnerGIS-Module erfolgreich geladen\")\n",
    "except ImportError as e:\n",
    "    print(f\"❌ FEHLER beim Import: {e}\")\n",
    "    print(\"💡 Tipp: Stelle sicher, dass du im Projekt-Root bist\")\n",
    "    raise\n",
    "\n",
    "# Settings\n",
    "warnings.filterwarnings('ignore')\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "%matplotlib inline\n",
    "\n",
    "# Display-Optionen\n",
    "pd.set_option('display.max_columns', None)\n",
    "pd.set_option('display.max_rows', 100)\n",
    "pd.set_option('display.float_format', '{:.2f}'.format)\n",
    "\n",
    "print(\"\\n🎨 Matplotlib & Pandas konfiguriert\")\n",
    "print(f\"📊 Pandas Version: {pd.__version__}\")\n",
    "print(f\"🐍 Python Version: {sys.version.split()[0]}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## ⚙️ Konfiguration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Config-Pfade (relativ zum Projekt-Root)\n",
    "cfg_paths = [\n",
    "    \"configs/base.yaml\",\n",
    "    \"configs/tech_catalog.yaml\",\n",
    "    \"configs/sites/default.site.yaml\",\n",
    "    \"configs/systems/baseline.system.yaml\",\n",
    "    \"configs/scenarios/pf_then_rh.scenario.yaml\",\n",
    "]\n",
    "\n",
    "# Prüfen ob Dateien existieren\n",
    "print(\"📋 Konfigurationsdateien:\")\n",
    "all_exist = True\n",
    "for p in cfg_paths:\n",
    "    full_path = project_root / p\n",
    "    exists = full_path.exists()\n",
    "    symbol = \"✅\" if exists else \"❌\"\n",
    "    print(f\"  {symbol} {p}\")\n",
    "    if not exists:\n",
    "        all_exist = False\n",
    "\n",
    "if not all_exist:\n",
    "    raise FileNotFoundError(\"❌ Nicht alle Config-Dateien gefunden!\")\n",
    "\n",
    "# Optional: Overrides für Quick-Tests\n",
    "overrides = None  # Oder z.B.: {\"run\": {\"solver\": \"glpk\"}}\n",
    "\n",
    "print(\"\\n✅ Konfiguration OK\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Config-Vorschau (optional)\n",
    "cfg_preview = load_and_merge(cfg_paths)\n",
    "\n",
    "print(\"🔍 Config-Vorschau:\")\n",
    "print(f\"  Solver:        {cfg_preview.get('run', {}).get('solver', 'N/A')}\")\n",
    "print(f\"  Zeitschritt:   {cfg_preview.get('run', {}).get('dt_h', 'N/A')} h\")\n",
    "print(f\"  CO2-Preis:     {cfg_preview.get('costs', {}).get('co2_price_eur_per_t', 'N/A')} EUR/t\")\n",
    "print(f\"  Input-Datei:   {cfg_preview.get('site', {}).get('input_xlsx', 'N/A')}\")\n",
    "print(f\"  Jahr:          {cfg_preview.get('site', {}).get('year_target', 'N/A')}\")\n",
    "\n",
    "# Systemkomponenten\n",
    "sys_cfg = cfg_preview.get('system', {})\n",
    "n_hp = len([hp for hp in sys_cfg.get('heat_pumps', []) if hp.get('enabled', True)])\n",
    "n_gen = len([k for k,v in sys_cfg.get('generators', {}).items() if v.get('enabled', False)])\n",
    "storage = sys_cfg.get('storage', {}).get('enabled', False)\n",
    "\n",
    "print(f\"\\n🏭 Systemkomponenten:\")\n",
    "print(f\"  Wärmepumpen:   {n_hp}\")\n",
    "print(f\"  Generatoren:   {n_gen}\")\n",
    "print(f\"  Speicher:      {'Ja' if storage else 'Nein'}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 🚀 Optimierung ausführen"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%time\n",
    "# Hauptlauf mit Error-Handling\n",
    "print(\"=\"*70)\n",
    "print(\"▶ STARTE OPTIMIERUNG\")\n",
    "print(\"=\"*70)\n",
    "print(f\"⏰ Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n\")\n",
    "\n",
    "try:\n",
    "    res = run_all(cfg_paths, overrides=overrides)\n",
    "    \n",
    "    print(\"\\n\" + \"=\"*70)\n",
    "    print(\"✅ OPTIMIERUNG ERFOLGREICH ABGESCHLOSSEN\")\n",
    "    print(\"=\"*70)\n",
    "    print(f\"\\n📊 Export:     {res['scenario_xlsx']}\")\n",
    "    print(f\"📁 Verzeichnis: {res['outdir']}\")\n",
    "    print(f\"\\n💰 Kosten:\")\n",
    "    for key, val in res['costs'].items():\n",
    "        print(f\"    {key:20s}: {val:,.2f}\")\n",
    "    \n",
    "    optimization_successful = True\n",
    "    \n",
    "except Exception as e:\n",
    "    print(\"\\n\" + \"=\"*70)\n",
    "    print(\"❌ FEHLER BEI DER OPTIMIERUNG\")\n",
    "    print(\"=\"*70)\n",
    "    print(f\"\\n🔴 Fehler: {str(e)}\\n\")\n",
    "    \n",
    "    import traceback\n",
    "    print(\"📋 Vollständiger Traceback:\")\n",
    "    traceback.print_exc()\n",
    "    \n",
    "    res = None\n",
    "    optimization_successful = False\n",
    "    \n",
    "    print(\"\\n💡 Troubleshooting:\")\n",
    "    print(\"  1. Prüfe ob Import_Data.xlsx existiert\")\n",
    "    print(\"  2. Prüfe Solver-Installation (gurobi/glpk)\")\n",
    "    print(\"  3. Prüfe ob alle Dependencies installiert sind\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 📊 Ergebnisse laden"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if optimization_successful and res:\n",
    "    # Excel laden\n",
    "    try:\n",
    "        ts = pd.read_excel(res['scenario_xlsx'], sheet_name='timeseries', index_col=0)\n",
    "        costs_df = pd.read_excel(res['scenario_xlsx'], sheet_name='costs', index_col=0)\n",
    "        meta_df = pd.read_excel(res['scenario_xlsx'], sheet_name='meta')\n",
    "        \n",
    "        print(\"✅ Daten erfolgreich geladen\")\n",
    "        print(f\"\\n📈 Zeitreihen:\")\n",
    "        print(f\"  Zeitschritte:  {len(ts):,}\")\n",
    "        print(f\"  Variablen:     {ts.shape[1]}\")\n",
    "        print(f\"  Zeitraum:      {ts.index.min()} bis {ts.index.max()}\")\n",
    "        \n",
    "        print(f\"\\n🔑 Verfügbare Spalten ({len(ts.columns)}):\")\n",
    "        for i, col in enumerate(sorted(ts.columns), 1):\n",
    "            print(f\"  {i:2d}. {col}\")\n",
    "        \n",
    "        data_loaded = True\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"❌ Fehler beim Laden: {e}\")\n",
    "        data_loaded = False\n",
    "else:\n",
    "    print(\"⚠️  Keine Ergebnisse zum Laden verfügbar\")\n",
    "    print(\"    Führe zuerst die Optimierung aus!\")\n",
    "    data_loaded = False"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 📈 Visualisierungen"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if data_loaded:\n",
    "    # Plot 1: Elektrische Leistung\n",
    "    fig, axes = plt.subplots(2, 1, figsize=(16, 10))\n",
    "    \n",
    "    # Strom\n",
    "    ax = axes[0]\n",
    "    ts[['P_buy', 'P_sell']].plot(ax=ax, linewidth=2, alpha=0.8)\n",
    "    ax.set_title('⚡ Elektrische Leistung', fontsize=16, fontweight='bold', pad=20)\n",
    "    ax.set_ylabel('Leistung [MW]', fontsize=12)\n",
    "    ax.set_xlabel('')\n",
    "    ax.grid(alpha=0.3, linestyle='--')\n",
    "    ax.legend(['Netzbezug', 'Einspeisung'], fontsize=11, loc='upper right')\n",
    "    \n",
    "    # Statistik einblenden\n",
    "    stats_text = f\"Peak: {ts['P_buy'].max():.1f} MW\\nTotal: {ts['P_buy'].sum():.0f} MWh\"\n",
    "    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, \n",
    "            fontsize=10, verticalalignment='top',\n",
    "            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))\n",
    "    \n",
    "    # Wärme\n",
    "    ax = axes[1]\n",
    "    heat_cols = [c for c in ts.columns if ('_Q' in c or '_Qth' in c) and c != 'Q_dump']\n",
    "    if heat_cols:\n",
    "        # Top 5 für Übersichtlichkeit\n",
    "        top_heat = ts[heat_cols].sum().nlargest(5).index.tolist()\n",
    "        ts[top_heat].plot(ax=ax, linewidth=2, alpha=0.8)\n",
    "        ax.set_title('🔥 Thermische Leistung (Top 5 Erzeuger)', fontsize=16, fontweight='bold', pad=20)\n",
    "    else:\n",
    "        ax.text(0.5, 0.5, 'Keine thermischen Daten gefunden', \n",
    "                ha='center', va='center', transform=ax.transAxes, fontsize=14)\n",
    "    \n",
    "    ax.set_ylabel('Leistung [MW_th]', fontsize=12)\n",
    "    ax.set_xlabel('Zeit', fontsize=12)\n",
    "    ax.grid(alpha=0.3, linestyle='--')\n",
    "    ax.legend(fontsize=10, loc='upper right')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "else:\n",
    "    print(\"⏭️  Überspringe Plots (keine Daten geladen)\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if data_loaded:\n",
    "    # Plot 2: Speicher & Heat Pumps\n",
    "    n_plots = 0\n",
    "    if 'TES_SOC' in ts.columns:\n",
    "        n_plots += 1\n",
    "    hp_cols = [c for c in ts.columns if c.startswith('HP') and '_Q' in c]\n",
    "    if hp_cols:\n",
    "        n_plots += 1\n",
    "    \n",
    "    if n_plots > 0:\n",
    "        fig, axes = plt.subplots(n_plots, 1, figsize=(16, 5*n_plots))\n",
    "        if n_plots == 1:\n",
    "            axes = [axes]\n",
    "        \n",
    "        plot_idx = 0\n",
    "        \n",
    "        # Speicher\n",
    "        if 'TES_SOC' in ts.columns:\n",
    "            ax = axes[plot_idx]\n",
    "            ts['TES_SOC'].plot(ax=ax, linewidth=2.5, color='orange', alpha=0.8)\n",
    "            ax.fill_between(ts.index, 0, ts['TES_SOC'], alpha=0.3, color='orange')\n",
    "            ax.set_title('🔋 Thermischer Speicher - State of Charge', \n",
    "                        fontsize=16, fontweight='bold', pad=20)\n",
    "            ax.set_ylabel('Energie [MWh]', fontsize=12)\n",
    "            ax.grid(alpha=0.3, linestyle='--')\n",
    "            \n",
    "            # Lade-/Entladezyklen zählen\n",
    "            if 'TES_Qc' in ts.columns and 'TES_Qd' in ts.columns:\n",
    "                cycles = ((ts['TES_Qc'] > 0).astype(int).diff().fillna(0) > 0).sum()\n",
    "                stats = f\"Zyklen: {cycles}\\nMax: {ts['TES_SOC'].max():.1f} MWh\"\n",
    "                ax.text(0.02, 0.98, stats, transform=ax.transAxes,\n",
    "                       fontsize=10, verticalalignment='top',\n",
    "                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))\n",
    "            plot_idx += 1\n",
    "        \n",
    "        # Heat Pumps\n",
    "        if hp_cols:\n",
    "            ax = axes[plot_idx]\n",
    "            ts[hp_cols].plot(ax=ax, linewidth=2, alpha=0.8)\n",
    "            ax.set_title('♨️  Wärmepumpen - Thermische Leistung', \n",
    "                        fontsize=16, fontweight='bold', pad=20)\n",
    "            ax.set_ylabel('Leistung [MW_th]', fontsize=12)\n",
    "            ax.set_xlabel('Zeit', fontsize=12)\n",
    "            ax.grid(alpha=0.3, linestyle='--')\n",
    "            ax.legend(fontsize=10, loc='upper right')\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.show()\n",
    "    else:\n",
    "        print(\"ℹ️  Keine Speicher- oder HP-Daten zum Plotten\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 📊 KPI-Analyse"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if data_loaded:\n",
    "    print(\"\\n\" + \"=\"*70)\n",
    "    print(\"📊 KEY PERFORMANCE INDICATORS\")\n",
    "    print(\"=\"*70)\n",
    "    \n",
    "    # Kosten\n",
    "    print(f\"\\n💰 Wirtschaftlichkeit:\")\n",
    "    print(f\"  Gesamtkosten:        {res['costs']['OBJ_value_EUR']:>15,.0f} EUR\")\n",
    "    print(f\"  Peak-Leistung:       {res['costs']['P_buy_peak_MW']:>15,.2f} MW\")\n",
    "    \n",
    "    # Energie-Bilanz\n",
    "    print(f\"\\n⚡ Elektrische Energie:\")\n",
    "    e_buy = ts['P_buy'].sum()\n",
    "    e_sell = ts['P_sell'].sum()\n",
    "    print(f\"  Netzbezug:           {e_buy:>15,.0f} MWh\")\n",
    "    print(f\"  Einspeisung:         {e_sell:>15,.0f} MWh\")\n",
    "    print(f\"  Netto:               {(e_buy - e_sell):>15,.0f} MWh\")\n",
    "    \n",
    "    # Wärme\n",
    "    if 'Q_dump' in ts.columns:\n",
    "        q_dump = ts['Q_dump'].sum()\n",
    "        print(f\"\\n🔥 Thermische Energie:\")\n",
    "        print(f\"  Wärme-Dump (Verlust): {q_dump:>14,.0f} MWh_th\")\n",
    "    \n",
    "    # Komponenten-Auslastung\n",
    "    print(f\"\\n🏭 Komponenten-Auslastung:\")\n",
    "    \n",
    "    # Heat Pumps\n",
    "    hp_cols = [c for c in ts.columns if c.startswith('HP') and c.endswith('_Q')]\n",
    "    if hp_cols:\n",
    "        print(f\"\\n  Wärmepumpen:\")\n",
    "        for col in sorted(hp_cols):\n",
    "            hp_id = col.replace('_Q', '')\n",
    "            avg = ts[col].mean()\n",
    "            max_val = ts[col].max()\n",
    "            hours_on = (ts[col] > 0.01).sum()\n",
    "            print(f\"    {hp_id:8s}: Ø {avg:6.2f} MW | Max {max_val:6.2f} MW | {hours_on:5d} h aktiv\")\n",
    "    \n",
    "    # Generatoren\n",
    "    gen_cols = [c for c in ts.columns if c.endswith('_Qth') and not c.startswith('HP')]\n",
    "    if gen_cols:\n",
    "        print(f\"\\n  Thermische Generatoren:\")\n",
    "        for col in sorted(gen_cols):\n",
    "            gen_id = col.replace('_Qth', '')\n",
    "            avg = ts[col].mean()\n",
    "            max_val = ts[col].max()\n",
    "            total = ts[col].sum()\n",
    "            print(f\"    {gen_id:8s}: Ø {avg:6.2f} MW | Max {max_val:6.2f} MW | {total:8,.0f} MWh_th\")\n",
    "    \n",
    "    # Speicher-Performance\n",
    "    if 'TES_SOC' in ts.columns:\n",
    "        print(f\"\\n  Speicher:\")\n",
    "        print(f\"    Max SOC:           {ts['TES_SOC'].max():>10.2f} MWh\")\n",
    "        print(f\"    Ø SOC:             {ts['TES_SOC'].mean():>10.2f} MWh\")\n",
    "        if 'TES_Qc' in ts.columns:\n",
    "            print(f\"    Total geladen:     {ts['TES_Qc'].sum():>10,.0f} MWh\")\n",
    "        if 'TES_Qd' in ts.columns:\n",
    "            print(f\"    Total entladen:    {ts['TES_Qd'].sum():>10,.0f} MWh\")\n",
    "    \n",
    "    print(\"\\n\" + \"=\"*70)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 🔍 Datenexploration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if data_loaded:\n",
    "    # DataFrame-Vorschau\n",
    "    print(\"📋 Zeitreihen-Daten (erste 10 Zeilen):\\n\")\n",
    "    display(ts.head(10))\n",
    "    \n",
    "    print(\"\\n📊 Statistische Zusammenfassung:\\n\")\n",
    "    display(ts.describe())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if data_loaded:\n",
    "    # Korrelationsmatrix (für Profis)\n",
    "    numeric_cols = ts.select_dtypes(include=[np.number]).columns\n",
    "    if len(numeric_cols) > 1:\n",
    "        fig, ax = plt.subplots(figsize=(12, 10))\n",
    "        corr = ts[numeric_cols].corr()\n",
    "        im = ax.imshow(corr, cmap='RdYlBu_r', aspect='auto', vmin=-1, vmax=1)\n",
    "        \n",
    "        ax.set_xticks(range(len(corr.columns)))\n",
    "        ax.set_yticks(range(len(corr.columns)))\n",
    "        ax.set_xticklabels(corr.columns, rotation=90, ha='right')\n",
    "        ax.set_yticklabels(corr.columns)\n",
    "        \n",
    "        plt.colorbar(im, ax=ax)\n",
    "        ax.set_title('🔗 Korrelationsmatrix', fontsize=16, fontweight='bold', pad=20)\n",
    "        plt.tight_layout()\n",
    "        plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 💾 Export & Weiterverarbeitung"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if data_loaded:\n",
    "    print(\"📁 Export-Informationen:\\n\")\n",
    "    print(f\"  Excel (komplett):  {res['scenario_xlsx']}\")\n",
    "    print(f\"  Verzeichnis:       {res['outdir']}\")\n",
    "    print(f\"\\n📄 Enthaltene Sheets:\")\n",
    "    print(f\"  - timeseries: Alle Zeitreihen-Variablen\")\n",
    "    print(f\"  - costs:      Kosten-Breakdown\")\n",
    "    print(f\"  - meta:       Config-Hash & Provenance\")\n",
    "    \n",
    "    # Optional: CSV-Export\n",
    "    csv_path = Path(res['outdir']) / 'timeseries.csv'\n",
    "    ts.to_csv(csv_path)\n",
    "    print(f\"\\n✅ CSV exportiert: {csv_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "## 🎯 Nächste Schritte\n",
    "\n",
    "### Sensitivitätsanalysen:\n",
    "- CO2-Preis variieren\n",
    "- Komponenten aktivieren/deaktivieren  \n",
    "- Kapazitäten anpassen\n",
    "\n",
    "### Weitere Analysen:\n",
    "- Jahresdauerlinie erstellen\n",
    "- Monats-Aggregation\n",
    "- COP-Entwicklung analysieren\n",
    "\n",
    "### Reporting:\n",
    "- Plots als PDF exportieren\n",
    "- Automatische Berichte generieren\n",
    "---"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.0"
  },
  "vscode": {
   "interpreter": {
    "hash": "your_env_hash_here"
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

NameError: name 'null' is not defined